## Cartpole exercise

#### Initiate environment

In [23]:
import gym
env = gym.make('CartPole-v1')

### Reinforcement learning

In [24]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam

In [25]:
states = env.observation_space.shape
actions = env.action_space.n

In [26]:
def build_model(states, actions):
    model = Sequential() 
    model.add(Input(states))
    model.add(Flatten())
    model.add(Dense(12, activation='relu', input_shape=(1,4)))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
model = build_model((1,4), actions)
model.build((1,4))

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 4)                 0         
                                                                 
 dense_6 (Dense)             (None, 12)                60        
                                                                 
 dense_7 (Dense)             (None, 12)                156       
                                                                 
 dense_8 (Dense)             (None, 2)                 26        
                                                                 
Total params: 242
Trainable params: 242
Non-trainable params: 0
_________________________________________________________________


In [29]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import WandbLogger

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=0.01), metrics=['mae'])

2022-02-16 18:41:33.610528: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:33.622941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:33.655948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:33.670004: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [32]:
dqn.fit(env, nb_steps=50000, callbacks=[WandbLogger(project="cart-pole")])

action_max,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
action_mean,▂▄▇▂▁▂▃▅▇▄██▅▄▅█▃▆▃▄▆▃▄▄▄▄▃▃▃▃▃▃▄▄▄▄▄▄▄▄
action_min,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
duration,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▇▁▃▄▂▃▂▄▃▂███▆▇
episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
episode_reward,▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃█▁▄▅▂▃▂▄▃▂███▆█
episode_steps,▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃█▁▄▅▂▃▂▄▃▂███▆█
loss,▁▁▁▁▂▂▂▄▄▄▄▅▄▅█▅▇▅▅▅
mae,▁▁▁▂▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇█
mean_q,▁▁▁▂▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇█
obs_max,▄▃▄▅▆▄▃▃▃▃▆▃▂▁▃▄▃▃▃▂▂▄▅▄▇▅▂▄▆▃▄▄█▆▅█▆▇▆▆


wandb: WARNING Upgrade to the 0.9.48 version of W&B Local to get the latest features. Learn more: http://wandb.me/local-upgrade


Training for 50000 steps ...
Interval 1 (0 steps performed)
   92/10000 [..............................] - ETA: 16s - reward: 1.0000

/Users/roberto/miniforge3/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2022-02-16 18:41:50.934824: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:50.947791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:50.954638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/roberto/miniforge3/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
 

  110/10000 [..............................] - ETA: 54s - reward: 1.0000

2022-02-16 18:41:51.393080: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:51.407615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:51.425688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-16 18:41:51.440896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10000/10000 [==============================] - 118s 12ms/step - reward: 1.0000
91 episodes - episode_reward: 109.462 [9.000, 418.000] - loss: 1.133 - mae: 18.355 - mean_q: 37.076

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 118s 12ms/step - reward: 1.0000
28 episodes - episode_reward: 346.036 [129.000, 500.000] - loss: 3.179 - mae: 36.653 - mean_q: 74.199

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 119s 12ms/step - reward: 1.0000
21 episodes - episode_reward: 483.952 [304.000, 500.000] - loss: 7.027 - mae: 47.905 - mean_q: 97.350

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 120s 12ms/step - reward: 1.0000
29 episodes - episode_reward: 336.966 [105.000, 500.000] - loss: 8.806 - mae: 50.827 - mean_q: 103.205

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 121s 12ms/step - reward: 1.0000
done, took 595.292 seconds


In [33]:
scores = dqn.test(env, nb_episodes=20, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 20 episodes ...
Episode 1: reward: 500.000, steps: 500
Episode 2: reward: 500.000, steps: 500
Episode 3: reward: 500.000, steps: 500
Episode 4: reward: 500.000, steps: 500
Episode 5: reward: 500.000, steps: 500
Episode 6: reward: 500.000, steps: 500
Episode 7: reward: 500.000, steps: 500
Episode 8: reward: 500.000, steps: 500
Episode 9: reward: 500.000, steps: 500
Episode 10: reward: 500.000, steps: 500
Episode 11: reward: 500.000, steps: 500
Episode 12: reward: 500.000, steps: 500
Episode 13: reward: 500.000, steps: 500
Episode 14: reward: 500.000, steps: 500
Episode 15: reward: 500.000, steps: 500
Episode 16: reward: 500.000, steps: 500
Episode 17: reward: 500.000, steps: 500
Episode 18: reward: 500.000, steps: 500
Episode 19: reward: 500.000, steps: 500
Episode 20: reward: 500.000, steps: 500
500.0


In [34]:
dqn.save_weights('cartpole_1.h5f', overwrite=True)

2022-02-16 18:53:27.449222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
